# Visualização dos microdados das matriculas da rede municipal de educação do município de SP

In [1]:
import pandas as pd

tabela = pd.read_csv('data/Microdados_EOL_Matriculas_Ano_2019.csv', encoding = "ISO-8859-1", sep = ';', chunksize=50000)
y = 0

for x in tabela:
    display(x)
    y = x
    break

# Dados faltando
y = y.drop("DESC_CICLO_ENSINO", axis=1)
y = y.drop("DESC_TIPO_PROGRAMA", axis=1)
y = y.drop("CD_PARECER_CONCL_FIN", axis=1)
y = y.drop("DESC_PARECER_CONCL_FIN", axis=1)

# Não consta no dicionário de variáveis
y = y.drop("DUR_DIA_HORA", axis=1)
y = y.drop("DUR_DIA_MIN", axis=1)
y = y.drop("X_SEMANA", axis=1)


,AN_LETIVO,CD_UNIDADE_EDUCACAO,NOME_DISTRITO,CD_SETOR,TIPO_ESCOLA,NOME_ESCOLA,DRE,CD_INEP_ESCOLA,SITUACAO_ESCOLA,CD_TURMA,...,SIT_AL_TRANSF,SIT_AL_FALEC,SIT_AL_NCOM,SIT_AL_EXCL,SIT_AL_CES,SIT_AL_REMAN,SIT_AL_RECLAS,SIT_AL_ATIVO,CD_PARECER_CONCL_FIN,DESC_PARECER_CONCL_FIN
0,2019,308258,MARSILAC,5201,CR.P.CONV,VOVO MARIAZINHA ROCHA,DIRETORIA REGIONAL DE EDUCACAO CAPELA DO SOCORRO,35571416.0,ATIVA,2023654,...,1,0,0,0,0,0,0,0,NaN,NaN
1,2019,309152,SAO MATEUS,7301,CR.P.CONV,NOAH,DIRETORIA REGIONAL DE EDUCACAO SAO MATEUS,35005592.0,ATIVA,2027168,...,0,0,0,0,0,0,0,0,NaN,NaN
2,2019,309152,SAO MATEUS,7301,CR.P.CONV,NOAH,DIRETORIA REGIONAL DE EDUCACAO SAO MATEUS,35005592.0,ATIVA,2027184,...,0,0,0,0,0,1,0,0,NaN,NaN
3,2019,700029,VILA CURUCA,8404,CEU AT COMPL,VILA CURUCA - IRENE RAMALHO,DIRETORIA REGIONAL DE EDUCACAO SAO MIGUEL,35455830.0,ATIVA,2092158,...,0,0,0,0,0,0,0,0,NaN,NaN
4,2019,308028,SAO MIGUEL,7405,CR.P.CONV,IRENE RAMALHO,DIRETORIA REGIONAL DE EDUCACAO SAO MIGUEL,35563067.0,ATIVA,2030404,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2019,308295,IGUATEMI,3302,CR.P.CONV,GALPAO ATRIA - LIMOEIRO I,DIRETORIA REGIONAL DE EDUCACAO SAO MATEUS,35577595.0,ATIVA,2026035,...,1,0,0,0,0,0,0,0,NaN,NaN
49996,2019,400791,VILA CURUCA,8408,CEI INDIR,NOVO CRUZEIRO,DIRETORIA REGIONAL DE EDUCACAO SAO MIGUEL,35495451.0,ATIVA,2082878,...,1,0,0,0,0,0,0,0,NaN,NaN
49997,2019,307572,GRAJAU,3012,CR.P.CONV,GUARANI,DIRETORIA REGIONAL DE EDUCACAO CAPELA DO SOCORRO,35457103.0,ATIVA,2011821,...,0,0,0,0,0,0,0,0,NaN,NaN
49998,2019,19502,ITAIM PAULISTA,3609,EMEI,"MARISA LOPES PINHEIRO, PROFA",DIRETORIA REGIONAL DE EDUCACAO SAO MIGUEL,35495748.0,ATIVA,2003358,...,0,0,0,0,0,0,0,0,NaN,NaN


In [10]:
y["DUR_DIA_TURMA"] = pd.to_numeric(y["DUR_DIA_TURMA"], errors="coerce")

y = y.dropna(how="any", axis=0)
y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41921 entries, 0 to 49999
Data columns (total 88 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   AN_LETIVO                 41921 non-null  int64  
 1   CD_UNIDADE_EDUCACAO       41921 non-null  int64  
 2   NOME_DISTRITO             41921 non-null  object 
 3   CD_SETOR                  41921 non-null  int64  
 4   TIPO_ESCOLA               41921 non-null  object 
 5   NOME_ESCOLA               41921 non-null  object 
 6   DRE                       41921 non-null  object 
 7   CD_INEP_ESCOLA            41921 non-null  float64
 8   SITUACAO_ESCOLA           41921 non-null  object 
 9   CD_TURMA                  41921 non-null  int64  
 10  CD_TURNO                  41921 non-null  int64  
 11  DESC_TURNO                41921 non-null  object 
 12  CD_SERIE                  41921 non-null  int64  
 13  DESC_SERIE                41921 non-null  object 
 14  MODALI

In [12]:

print(y[:, 5])

TypeError: '(slice(None, None, None), 5)' is an invalid key

In [14]:
from wordcloud import WordCloud
text = y['NOME_ESCOLA'].str.cat(sep=' ')
print(text)


VOVO MARIAZINHA ROCHA NOAH NOAH IRENE RAMALHO UNIVERSO DA CRIANCA III JARDIM NOVO PARELHEIROS SAO LUCAS JARDIM DA VOVO ANA BETEL VILA NOVA CURUCA JESUS MENINO,C. ED. COMUNITARIO SANTA IZABEL I CANTINHO FELIZ VILA LEOPOLDINA RENASCER RENASCER JOAQUIM THOME FILHO, VER. PAULINOEVA DORINA NOWILL, PROFESSORA CAMINHAR E PRECISO II DOMINGOS  RUFINO DE SOUZA ALPS LAJEADO II SMILES PICOLO CONJ. HABITACIONAL SAO FRANCISCO MANOEL DE ALVARENGA FREIRE JUNIOR PROF CARMEM MIRANDA DIVINA GRACA ESTRELA AZUL CRESCENDO PARA BRILHAR EDUCAR SAO NORBERTO PAULO DUARTE PAULO DUARTE EDUCARTE I PENTAGRAMA I MURURES ALVARENGA MARINA NOGUEIRA DE SOUZA MARTINS ZULMIRA CAVALHEIRO FAUSTINO JORGINHO RECANTO INFANTIL COMUNITARIO DO PQ FIGUEIRA GRANDE CAROLINA RENNO RIBEIRO DE OLIVEIRA, PROFA. CAROLINA RENNO RIBEIRO DE OLIVEIRA, PROFA. REINO DA CRIANCA II ERMELINO MATARAZZO, CRECHE PRE ERMELINO MATARAZZO, CRECHE PRE INEZ MENEZES MARIA TAMANDARE, ALM. TAMANDARE, ALM. JARDIM DOS ANJOS EDUCARTE I PENHA EFRAIM CEDRINHO III